In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy.linalg as la
import scipy.sparse as sparse
from matplotlib import cm
import timeit
import numba
from numba import jit

In [2]:
#output settings
np.set_printoptions(precision=2)
%matplotlib qt 

In [3]:
def u_analytical(x, y):

    uan = np.sin(np.pi*x)*np.sin(np.pi*y) + x

    return uan

def source(x, y):

    f = -2*np.pi**2*np.sin(np.pi*x)*np.sin(np.pi*y)

    return f

In [4]:
@jit(nopython=True)
def sor_iteration(matrix, source, grid, init_guess=None, boundary=((0, 0), (0,0)), tolerance=1.e-8, itermax=1000, omega=1.5):

    M, f, u0, B, tol, kmax, w = matrix, source, init_guess, boundary, tolerance, itermax, omega

    x, y = grid

    Nx, Ny = x.shape[1]-1, y.shape[0]-1

    dx, dy = (x[0, -1] - x[0, 0])/Nx, (y[-1, 0] - y[0, 0])/Ny

    if init_guess is None:

        u0 = np.ones_like(f)

    #assign dirichlet boundary conditions
    u0[:, 0] = B[0][0]                               
    u0[:, -1] = B[0][1]
    u0[0, :] = B[1][0]
    u0[-1, :] = B[1][1]
    
    u = u0
    k = 0
    rel_diff = tol + 1
    conv_hist = []
       
    #iteration loop using nested loops.
    while  k < kmax and rel_diff > tol:

        u_next = u.copy()

        for j in range(1, Ny):
            for i in range(1, Nx):
                
                u_next[j, i] = (1-omega)*u[j,i] + omega*((u_next[j, i-1] + u[j, i+1])*dy**2 + 
                                (u_next[j-1, i] + u[j+1, i])*dx**2 - f[j,i]*dx**2*dy**2)/(2*(dx**2+dy**2))
        
        

        rel_diff = la.norm(u_next-u)/la.norm(u)
        conv_hist.append(rel_diff)
        u = u_next
        k += 1

    return u, k, rel_diff, conv_hist

In [5]:
Nx = 40
Ny = 40

x_I, x_F = 0, 1
y_I, y_F = 0, 1

X = np.linspace(x_I, x_F, Nx+1)
Y = np.linspace(y_I, y_F, Ny+1)

x, y = np.meshgrid(X, Y)

In [6]:
#Set up the solver parameters

f = source(x, y)                                          #source term

u_guess = np.sin(2*np.pi*x)*np.sin(2*np.pi*y)             #initial guess

itermax = 2000000                                         #maximum number of iterations

tol = 1e-8                                                #desired tolerance

omega = 1.5  #2/(1+np.sin(np.pi/Nx))

boundary = ((0,1), (X, X))                                #dirichlet boundary conditions
# print(omega_opt)

In [7]:
#Test the solver


start = timeit.default_timer()            #time the solver

u, iternum, rel_diff, conv_hist = sor_iteration(np.eye(Nx), f, (x, y), init_guess=None, 
                                                boundary=boundary, tolerance=tol, itermax=itermax, omega=omega)       #calculate the solution

end = timeit.default_timer()

elapsed = (end - start)

u_an = u_analytical(x, y)                #analytical solution for comparison

if iternum == itermax:

    print('WARNING: desired tolerance has not been reached for the given maximum iterations \n')

print('Omega = {:1.2E}'
      'Number of iterations: {} \n'
      'Last relative difference: {:1.4e} \n'
      'Time elapsed: {:1.2e} s'.format(omega, iternum, rel_diff, elapsed))

Omega = 1.50E+00Number of iterations: 665 
Last relative difference: 9.8448e-09 
Time elapsed: 2.37e+00 s


In [8]:
u_an = u_analytical(x, y)

error = la.norm(u - u_an, 2)/la.norm(u_an, 2)

print(error)

0.00025635664449666536


In [9]:
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')


scat = ax.scatter(x, y, u, c = 'r', label = 'numerical', alpha = 1)
surf = ax.plot_surface(x, y, u_an, cmap = cm.coolwarm, label = 'analytical', alpha = 0.3)

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('$u = \sin\pi x \sin\pi y + x$')
# ax.set_zlim(top=2., bottom=-0.5)

surf._facecolors2d=surf._facecolors3d
surf._edgecolors2d=surf._edgecolors3d
ax.legend(loc = 2)
plt.show()

TypeError: numpy_jacobi() missing 3 required positional arguments: 'matrix', 'source', and 'grid'